In [1]:
#Demonstration
import math
    
def decoratorlog(f):
    def wrapper(*args):
        result =  f(*args)
        print(f"{f.__name__} de {args} est: {result}") 
        return result
    return wrapper
    
#@decoratorlog
def calculer_entropie(probabilites):
    entropie = 0
    for p in probabilites:
        if p > 0:
            entropie -= p * math.log(p, 2)
    return entropie

# Exemple d'utilisation
probabilites = [0.5, 0.5]  # Probabilités pour deux classes équilibrées
entropie = calculer_entropie(probabilites)


In [2]:
probabilites = [0.3, 0.3, 0.3]  # Probabilités pour deux classes équilibrées
entropie = calculer_entropie(probabilites)

In [3]:
probabilites = [0.4, 0.2, 0.3]  # Probabilités pour deux classes équilibrées
entropie = calculer_entropie(probabilites)

In [4]:
probabilites = [0.6, 0.2, 0.2]  # Probabilités pour deux classes équilibrées
entropie = calculer_entropie(probabilites)

In [5]:
probabilites = [0.6, 0.1, 0.3]  # Probabilités pour deux classes équilibrées
entropie = calculer_entropie(probabilites)

In [6]:
calculer_entropie([0.333, 0.6666]) + calculer_entropie([1])

0.9183053483987162

In [7]:
calculer_entropie([0.5, 0.5]) + calculer_entropie([1])

1.0

In [8]:
#@decoratorlog
def createDataSet(*args):
    dataSet = [[1, 1, 'yes'],
         [1, 1, 'yes'],
         [1, 0, 'no'],
         [0, 1, 'no'],
         [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    return dataSet, labels
dataSet, labels = createDataSet()

In [9]:
print(dataSet[:])
print(dataSet[:][0])
print(dataSet[:][0][2])

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
[1, 1, 'yes']
yes


In [10]:
from math import log
#@decoratorlog
def calcShannonEnt(dataSet):
     numEntries = len(dataSet)
     labelCounts = {}
     for featVec in dataSet: 
         currentLabel = featVec[-1] 
         if currentLabel not in labelCounts.keys():
             labelCounts[currentLabel] = 0 
         labelCounts[currentLabel] += 1 
     shannonEnt = 0.0
     for key in labelCounts:
         prob = float(labelCounts[key])/numEntries
         shannonEnt -= prob * log(prob,2) 
     return shannonEnt
    
calcShannonEnt(dataSet)
calcShannonEnt([[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no'],[0, 1, 'yes']])
calcShannonEnt([[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no'],[0, 1, 'no']])
calcShannonEnt([[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no'],[0, 1, 'no'], [0, 1, 'no']])
calcShannonEnt([[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no'],[0, 1, 'maybe']])


1.4591479170272448

In [11]:
#Splitting Dataset
#@decoratorlog
def splitDataSet(dataSet, axis, value):
    retDataset =[]
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataset.append(reducedFeatVec)
    return retDataset
dataSet, labels = createDataSet()
splitDataSet(dataSet, 0, 1)
splitDataSet(dataSet, 0, 0)
splitDataSet(dataSet, 1, 0)
splitDataSet(dataSet, 1, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]

In [12]:
print(dataSet[0])
dataSet, labels = createDataSet()
#@decoratorlog
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList) 
        newEntropy = 0.0
        for value in uniqueVals:
            subDataset = splitDataSet(dataSet, i, value)
            prob = len(subDataset) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataset)
        infoGain = baseEntropy - newEntropy 
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain 
            bestFeature = i 
    return bestFeature 

chooseBestFeatureToSplit(dataSet)   
    
    

[1, 1, 'yes']


0

In [13]:
import operator
#@decoratorlog
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]
    
majorityCnt(['yes', 'yes', 'no'])
majorityCnt(['yes', 'yes', 'no','no','maybe'])
majorityCnt(['yes', 'yes', 'no','no','no'])

'no'

In [16]:
#@decoratorlog
def createTree(dataSet, labels):
     classList = [example[-1] for example in dataSet]
     if classList.count(classList[0]) == len(classList):
         return classList[0] 
     if len(dataSet[0]) == 1: 
         return majorityCnt(classList) 
     bestFeat = chooseBestFeatureToSplit(dataSet)
     bestFeatLabel = labels[bestFeat]
     myTree = {bestFeatLabel:{}}
     del(labels[bestFeat]) 
     featValues = [example[bestFeat] for example in dataSet] 
     uniqueVals = set(featValues) 
     for value in uniqueVals:
         subLabels = labels[:] 
         myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
     return myTree
dataSet, labels = createDataSet()
createTree(dataSet, labels)


{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [26]:
dataSet, labels = createDataSet()
label_input = labels[:]
tree = createTree(dataSet, label_input)

#@decoratorlog
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr) 
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__=='dict':
                classLabel = classify(secondDict[key],featLabels,testVec)
            else: classLabel = secondDict[key]
    return classLabel
classify(tree, labels, [1, 0])

'no'

In [32]:
import pickle

# Créons un dictionnaire exemple
data = {'a': [1, 2.0, 3, 4+6j],
        'b': ('string', u'Unicode string'),
        'c': None}

# Sérialisons et stockons l'objet dans un fichier
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)
with open('data.pickle', 'rb') as f:
    data_loaded = pickle.load(f)

print(data_loaded)

{'a': [1, 2.0, 3, (4+6j)], 'b': ('string', 'Unicode string'), 'c': None}


In [35]:
import pickle
def storeTree(inputTree, filename):
     fw = open(filename,'wb')
     pickle.dump(inputTree, fw)
     fw.close()
 
def grabTree(filename):
    fr = open(filename, 'rb')
    return pickle.load(fr)
print(tree)
storeTree(tree, "./checkpoints.pt")
print(grabTree("./checkpoints.pt"))

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
